# Usando o algoritmo XGBoost to prever quando uma empresa irá perder um cliente

## Ambiente recomendado

- Image: Data Science
- Kernel: Python 3
- Instance type: ml.t3.medium

## Background

Esse notebook é utilizado para treinar um modelo que prevê as chances de uma empresa perder um cliente. As entradas são os dados do cliente de um provedor de celular, com o objetivo 'Churn?' que diz se o cliente abandonou ou não a empresa. Esse notebook é adaptado dos [exemplos com SageMaker](https://github.com/aws/amazon-sagemaker-examples/blob/main/introduction_to_applying_machine_learning/xgboost_customer_churn/xgboost_customer_churn.ipynb)

## Inicializando o Ambiente e variáveis globais

In [ ]:
# Import libraries
import boto3
import re
import pandas as pd
import numpy as np
import os

import sagemaker
from sagemaker import get_execution_role
from sagemaker.serializers import CSVSerializer
from sagemaker.inputs import TrainingInput

# Get the SageMaker session and the execution role from the SageMaker domain
sess = sagemaker.Session()
role = get_execution_role()

bucket = '<name-of-your-bucket>' # Update with the name of a bucket that is already created in S3
prefix = 'demo-xgboost' # The name of the folder that will be created in the S3 bucket

---
## Dataset

Para essa atividade, os dados já foram preparados e também separados em dois arquivos: `train.csv` (usado para treinar o modelo) e `validation.csv` (usado para validar como o modelo se porta).


Vamos configurar os arquivos locais e fazer upload para o bucket S3, para que o SageMaker possa utilizar esses dados no processamento de jobs.

In [ ]:
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation/validation.csv')).upload_file('validation.csv')

---
## Treinamento

Agora que nós temos nossos dados armazenados no S3, nós podemos começar o treinamento. Nessa seção, devemos especificar três coisas importantes: onde nossos dados de treinamento estão, o caminho para o container do algoritmo (ECR) que irá montar o modelo, e o algoritmo que vamos utilizar (junto com seus hiperparâmetros). Mais informações sobre o algoritmo XGBoost podem ser encontrados [aqui](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost_hyperparameters.html).


In [ ]:
# The location of our training and validation data in S3
s3_input_train = TrainingInput(
    s3_data='s3://{}/{}/train'.format(bucket, prefix), content_type='csv'
)
s3_input_validation = TrainingInput(
    s3_data='s3://{}/{}/validation/'.format(bucket, prefix), content_type='csv'
)

In [ ]:
# The location of the XGBoost container version 1.5-1 (an AWS-managed container)
container = sagemaker.image_uris.retrieve('xgboost', sess.boto_region_name, '1.5-1')

In [ ]:
# Initialize hyperparameters
hyperparameters = {
                    'max_depth':'5',
                    'eta':'0.2',
                    'gamma':'4',
                    'min_child_weight':'6',
                    'subsample':'0.8',
                    'objective':'binary:logistic',
                    'eval_metric':'error',
                    'num_round':'100'}

# Output path where the trained model will be saved
output_path = 's3://{}/{}/output'.format(bucket, prefix)

# Set up the Estimator, which is training job
xgb = sagemaker.estimator.Estimator(image_uri=container, 
                                    hyperparameters=hyperparameters,
                                    role=role,
                                    instance_count=1, 
                                    instance_type='ml.m4.xlarge', 
                                    output_path=output_path,
                                    sagemaker_session=sess)

In [ ]:
# "fit" executes the training job
xgb.fit({'train': s3_input_train, 'validation': s3_input_validation}) 